In [1]:
import pandas as pd
import numpy as np
import re
from pandas import DataFrame
from pandas.api.types import is_numeric_dtype, is_categorical_dtype

In [2]:
data=pd.read_csv(filepath_or_buffer=r'C:/Users/Abhishek/Downloads/testing.csv')
data.head()

,timestamp,work_item,work_type,work_priority,domain,platform,components,from_phase,to_phase,from_resource,to_resource
0,02-01-2015 14:39,WI_000001,WT_00001,WP_00003,BD_00001,PL_00001,NaN,NaN,Analyze,NaN,ER_00043
1,02-01-2015 15:04,WI_000002,WT_00002,WP_00004,BD_00002,PL_00002,CO_00037/CO_00071,NaN,Analyze,NaN,ER_00225
2,02-01-2015 15:28,WI_000003,WT_00002,WP_00004,BD_00002,PL_00002,CO_00037/CO_00071,NaN,Analyze,NaN,ER_00225
3,02-01-2015 15:33,WI_000004,WT_00002,WP_00004,BD_00002,PL_00002,CO_00037/CO_00071,NaN,Analyze,NaN,ER_00225
4,02-01-2015 16:32,WI_000005,WT_00002,WP_00003,BD_00003,PL_00003,CO_00113,NaN,Analyze,NaN,ER_00206


In [8]:
#helper functions
cont_names, cat_names, bool_names = [],[],[]
def cont_cat_split(df, max_card=20, dep_var=None):
    "Helper function that returns column names of cont and cat variables from given df."
    cont_names, cat_names, bool_names = [], [], []
    for label in df:
        if label == dep_var: continue
        if len(set(df[label])) > max_card and df[label].dtype == np.int64 or df[label].dtype == float: cont_names.append(label)
        if df[label].dtype == object: cat_names.append(label)
        if df[label].dtype == bool: bool_names.append(label)
    return cont_names, cat_names, bool_names

#cont_names, cat_names, bool_names = cont_cat_split(data,1)


def catogorical_convert(df:DataFrame, cat_names:list=cat_names):
        "Transform `self.cat_names` columns in categorical."
        categories = {}
        for n in cat_names:
            df.loc[:,n] = df.loc[:,n].astype('category').cat.as_ordered()
            categories[n] = df[n].cat.categories
        return df
    


def make_date(df:DataFrame, date_field:str):
    "Make sure `df[field_name]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True,
                                       format='%d-%m-%y %H:%M',dayfirst =True)
def ifnone(a,b):
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a


def add_datepart(df:DataFrame, field_name:str, prefix:str=None, drop:bool=True, time:bool=False):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."
    make_date(df, field_name)
    field = df[field_name]
    prefix = ifnone(prefix, re.sub('[Dd]ate$', '', field_name))
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[prefix + n] = getattr(field.dt, n.lower())
    df[prefix + 'Elapsed'] = field.astype(np.int64) // 10 ** 9
    if drop: df.drop(field_name, axis=1, inplace=True)
    return df



def cat_to_numerical(df:DataFrame):
    for i in df:
        if is_categorical_dtype(df[i].dtypes):df[i]=df[i].cat.codes
    return df




In [9]:
data['timestamp']=pd.to_datetime(data['timestamp'],format='%d-%m-%y %H:%M',infer_datetime_format=True,dayfirst =True)
data=add_datepart(data,'timestamp',drop=False,time=True)
#data.head().T
data.dtypes


timestamp                    datetime64[ns]
work_item                            object
work_type                            object
work_priority                        object
domain                               object
platform                             object
components                           object
from_phase                           object
to_phase                             object
from_resource                        object
to_resource                          object
timestampYear                         int64
timestampMonth                        int64
timestampWeek                         int64
timestampDay                          int64
timestampDayofweek                    int64
timestampDayofyear                    int64
timestampIs_month_end                  bool
timestampIs_month_start                bool
timestampIs_quarter_end                bool
timestampIs_quarter_start              bool
timestampIs_year_end                   bool
timestampIs_year_start          

In [10]:
cont_names, cat_names, bool_names = cont_cat_split(data,1)
print(cont_names, cat_names, bool_names)

data=catogorical_convert(data,cont_names)
data=catogorical_convert(data,cat_names)
data.dtypes

['timestampYear', 'timestampMonth', 'timestampWeek', 'timestampDay', 'timestampDayofweek', 'timestampDayofyear', 'timestampHour', 'timestampMinute', 'timestampElapsed'] ['work_item', 'work_type', 'work_priority', 'domain', 'platform', 'components', 'from_phase', 'to_phase', 'from_resource', 'to_resource'] ['timestampIs_month_end', 'timestampIs_month_start', 'timestampIs_quarter_end', 'timestampIs_quarter_start', 'timestampIs_year_end', 'timestampIs_year_start']


timestamp                    datetime64[ns]
work_item                          category
work_type                          category
work_priority                      category
domain                             category
platform                           category
components                         category
from_phase                         category
to_phase                           category
from_resource                      category
to_resource                        category
timestampYear                      category
timestampMonth                     category
timestampWeek                      category
timestampDay                       category
timestampDayofweek                 category
timestampDayofyear                 category
timestampIs_month_end                  bool
timestampIs_month_start                bool
timestampIs_quarter_end                bool
timestampIs_quarter_start              bool
timestampIs_year_end                   bool
timestampIs_year_start          

In [ ]:
data.work_type.cat.categories



cat_szs = [len(data[n].cat.categories)+1 for n in cat_names]
emb_szs = [(i, min(600, round(i*1.6**0.56))) for i in cat_szs]
emb_szs